Importing the relevant libraries.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
from tensorflow.keras.callbacks import TensorBoard
import time

Loading dataset and setting inital variables

In [2]:
model_name='perceptron'
dataset_name="normal_distribution_dataset"
validation_split=0.2
total_epoch=50
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

Splitting dataset into training and testing

In [3]:
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=validation_split, random_state=42)

Defining and compiling the model

In [4]:
# Define the model
dset_model_Perceptron = tf.keras.Sequential([
    layers.Dense(1)
])

# Compile the model
dset_model_Perceptron.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Define TensorBoard callback
tensorboard_callback = TensorBoard(log_dir='./../../Observation/'+model_name+'/'+model_name+dataset_name+'logs')

Training the model

In [5]:
# Start timer for training time
start_time = time.time()

# Train the model with TensorBoard callback
history = dset_model_Perceptron.fit(X_train, y_train, epochs=total_epoch, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])

# End timer for training time
training_time = time.time() - start_time

Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4935 - loss: 0.7121 - val_accuracy: 0.5063 - val_loss: 0.3360
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4997 - loss: 0.3045 - val_accuracy: 0.5040 - val_loss: 0.2548
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5008 - loss: 0.2525 - val_accuracy: 0.4952 - val_loss: 0.2501
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5028 - loss: 0.2501 - val_accuracy: 0.4990 - val_loss: 0.2500
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5044 - loss: 0.2500 - val_accuracy: 0.5033 - val_loss: 0.2500
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 988us/step - accuracy: 0.5049 - loss: 0.2501 - val_accuracy: 0.5000 - val_loss: 0.2500
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5013 - loss: 0.2500 - val_accuracy: 0.4995 - val_loss: 0.2501
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4942 - loss: 0.2501 - val_accuracy: 

Making prediction and calculating the metrics

In [6]:
# Make predictions
predictions = dset_model_Perceptron.predict(X_test)

# Calculate metrics
# Calculate accuracy with threshold
threshold = 0.5  # Define the threshold
correct_predictions = np.sum(np.abs(y_test - predictions.flatten()) <= threshold)
total_predictions = len(y_test)
accuracy = correct_predictions / total_predictions  # Final accuracy
mse = mean_squared_error(y_test, predictions)  # Mean Squared Error
mae = mean_absolute_error(y_test, predictions)  # Mean Absolute Error
rmse = np.sqrt(mse)  # Root Mean Squared Error
f1 = f1_score(y_test, predictions.round(), average='micro')  # F1 Score

# Calculate inference time
start_time = time.time()
predictions = dset_model_Perceptron.predict(X_test)
inference_time = time.time() - start_time

# Print and store metrics
metrics = {
    'Model type': model_name,
    'dataset': dataset_name,
    'Accuracy': accuracy,
    'MSE': mse,
    'MAE': mae,
    'RMSE': rmse,
    'F1 Score': f1,
    'Training Time': training_time,
    'Inference Time': inference_time
}

print("Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value}")

# Store metrics in a list
metrics_list = list(metrics.values())
pd.DataFrame([metrics_list], columns=metrics.keys()).to_csv('./../../Observation/'+model_name+'/'+model_name+dataset_name+'metrics.csv', index=False)

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 441us/step
Metrics:
Model type: perceptron
dataset: normal_distribution_dataset
Accuracy: 0.4995
MSE: 0.2500066994866024
MAE: 0.4999477005377412
RMSE: 0.5000066994417199
F1 Score: 0.4995
Training Time: 30.351420402526855
Inference Time: 0.1191561222076416


Making a function to generate models for different dataset

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
from tensorflow.keras.callbacks import TensorBoard
import time

def configure_model(layers_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy']):
    model = tf.keras.Sequential()

    for layer_config in layers_config:
        neurons, layer_type = layer_config
        if layer_type == 'dense':
            model.add(layers.Dense(neurons))
        elif layer_type == 'dropout':
            model.add(layers.Dropout(neurons))
        elif layer_type == 'activation':
            model.add(layers.Activation(neurons))

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

def train_model(model, X_train, y_train, X_test, y_test, model_name='perceptron', dataset_name="addition_dataset_2", total_epoch=50):
    tensorboard_callback = TensorBoard(log_dir='./../../Observation/'+model_name+'/'+model_name+dataset_name+'logs')

    start_time = time.time()

    history = model.fit(X_train, y_train, epochs=total_epoch, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])

    training_time = time.time() - start_time

    predictions = model.predict(X_test)

    return model, history, predictions, training_time

def test_model(model, X_test, y_test, model_name='perceptron', dataset_name="addition_dataset_2"):
    start_time = time.time()
    predictions = model.predict(X_test)
    inference_time = time.time() - start_time
    
    metrics = calculate_metrics(predictions, y_test)
    metrics['Training Time'] = training_time
    metrics['Inference Time'] = inference_time
    metrics['Model type'] = model_name
    metrics['dataset'] = dataset_name

    print("Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

    metrics_list = list(metrics.values())
    pd.DataFrame([metrics_list], columns=metrics.keys()).to_csv('./../../Observation/'+model_name+'/'+model_name+dataset_name+'metrics.csv', index=False)

    return metrics

def calculate_metrics(predictions, true_labels):
    threshold = 0.5  # Define the threshold
    correct_predictions = np.sum(np.abs(true_labels - predictions.flatten()) <= threshold)
    total_predictions = len(true_labels)
    accuracy = correct_predictions / total_predictions  # Final accuracy
    mse = mean_squared_error(true_labels, predictions)  # Mean Squared Error
    mae = mean_absolute_error(true_labels, predictions)  # Mean Absolute Error
    rmse = np.sqrt(mse)  # Root Mean Squared Error
    f1 = f1_score(true_labels.round(), predictions.round(), average='micro')  # F1 Score

    return {
        'Accuracy': accuracy,
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse,
        'F1 Score': f1
    }

In [8]:
# Load dataset
dataset_name = "binomial_distribution_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
perceptron_config = [(1, 'dense')]
model = configure_model(perceptron_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,dataset_name=dataset_name)

# Test the model
test_metrics = test_model(model, X_test, y_test,dataset_name=dataset_name)

Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5087 - loss: 2.0803 - val_accuracy: 0.4897 - val_loss: 0.2760
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5038 - loss: 0.2593 - val_accuracy: 0.4972 - val_loss: 0.2537
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5065 - loss: 0.2520 - val_accuracy: 0.4972 - val_loss: 0.2531
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5066 - loss: 0.2521 - val_accuracy: 0.4863 - val_loss: 0.2530
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5052 - loss: 0.2515 - val_accuracy: 0.4972 - val_loss: 0.2519
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5072 - loss: 0.2508 - val_accuracy: 0.4863 - val_loss: 0.2522
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5086 - loss: 0.2508 - val_accuracy: 0.4972 - val_loss: 0.2510
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5164 - loss: 0.2499 - val_accuracy: 0.

In [9]:
# Load dataset
dataset_name = "exponential_distribution_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
perceptron_config = [(1, 'dense')]
model = configure_model(perceptron_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,dataset_name=dataset_name)

# Test the model
test_metrics = test_model(model, X_test, y_test,dataset_name=dataset_name)

Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5015 - loss: 8.9933 - val_accuracy: 0.4972 - val_loss: 3.6919
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step - accuracy: 0.5052 - loss: 2.7504 - val_accuracy: 0.4935 - val_loss: 1.0084
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5073 - loss: 0.7778 - val_accuracy: 0.4972 - val_loss: 0.4005
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 944us/step - accuracy: 0.5039 - loss: 0.3679 - val_accuracy: 0.5010 - val_loss: 0.3055
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 976us/step - accuracy: 0.4967 - loss: 0.2991 - val_accuracy: 0.5023 - val_loss: 0.2786
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 972us/step - accuracy: 0.4921 - loss: 0.2747 - val_accuracy: 0.5033 - val_loss: 0.2630
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4977 - loss: 0.2605 - val_accuracy: 0.5010 - val_loss: 0.2546
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4979 - loss: 0.2544 - val_accu

In [10]:
# Load dataset
dataset_name = "gamma_distribution_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
perceptron_config = [(1, 'dense')]
model = configure_model(perceptron_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,dataset_name=dataset_name)

# Test the model
test_metrics = test_model(model, X_test, y_test,dataset_name=dataset_name)

Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4983 - loss: 47.5238 - val_accuracy: 0.4983 - val_loss: 24.6826
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4957 - loss: 20.8440 - val_accuracy: 0.4967 - val_loss: 9.5362
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5036 - loss: 7.8950 - val_accuracy: 0.4955 - val_loss: 3.4252
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5020 - loss: 2.8473 - val_accuracy: 0.5008 - val_loss: 1.5717
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4982 - loss: 1.4475 - val_accuracy: 0.4972 - val_loss: 1.1206
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4975 - loss: 1.0892 - val_accuracy: 0.4972 - val_loss: 0.9424
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5041 - loss: 0.9011 - val_accuracy: 0.4963 - val_loss: 0.7903
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4982 - loss: 0.7551 - val_accuracy:

In [11]:
# Load dataset
dataset_name = "poisson_distribution_dataset"
d_set = pd.read_csv('./../../Data/'+dataset_name+'.csv')
dset_features = d_set.copy()
dset_labels = dset_features.pop('result')
dset_features = np.array(dset_features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(dset_features, dset_labels, test_size=0.2, random_state=42)

# Configure the model with multiple layers
#layers_config = [(32, 'dense'), (0.2, 'dropout'), ('relu', 'activation')]
perceptron_config = [(1, 'dense')]
model = configure_model(perceptron_config, optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model, history, predictions, training_time = train_model(model, X_train, y_train, X_test, y_test,dataset_name=dataset_name)

# Test the model
test_metrics = test_model(model, X_test, y_test,dataset_name=dataset_name)

Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4918 - loss: 13.4214 - val_accuracy: 0.4940 - val_loss: 5.2470
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4962 - loss: 3.9032 - val_accuracy: 0.4918 - val_loss: 1.2567
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4979 - loss: 0.9351 - val_accuracy: 0.4835 - val_loss: 0.3971
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4929 - loss: 0.3477 - val_accuracy: 0.5030 - val_loss: 0.2960
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5051 - loss: 0.2896 - val_accuracy: 0.5030 - val_loss: 0.2834
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4977 - loss: 0.2810 - val_accuracy: 0.5030 - val_loss: 0.2753
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 992us/step - accuracy: 0.5044 - loss: 0.2723 - val_accuracy: 0.5030 - val_loss: 0.2676
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4983 - loss: 0.2666 - val_accuracy: